In [1]:
import re
import googlemaps
from datetime import datetime
import pandas as pd
import requests
import re
import os
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_apiKey")


True

In [2]:
def getfromGooleApi(queryParams):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    res = requests.get(url, params=queryParams)
    print(res.status_code)
    return res.json()

In [3]:
#pongo la info de Seatle
def Querys(radius,tipo,keyword,pagetoken):
    return {"location":"47.60621,-122.33207",
    "radius":radius,
    "type":tipo,
    "keyword":keyword,
    "key":f'{apiKey}',
    "pagetoken":pagetoken}

In [4]:
def getgeo(res,check,name):
    lista_local = []
    for e in res["results"]:
        if re.search(check, e["name"]):
            lista_local.append({"category":name,"name":e["name"],
                           "location":{"type":"Point","coordinates":[e["geometry"]["location"]["lng"],e["geometry"]["location"]["lat"]]}})
    return lista_local

In [5]:
airport = getfromGooleApi(Querys(17500,"airport","SEA",None))
airport

200


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 47.4502499, 'lng': -122.3088165},
    'viewport': {'northeast': {'lat': 47.45462985000001, 'lng': -122.2911378},
     'southwest': {'lat': 47.43711004999999, 'lng': -122.3147094}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/airport-71.png',
   'id': '2eea81c16dbf0dcc0452910206105d3d1b1484ac',
   'name': 'Seattle-Tacoma International Airport',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3456,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104238425351880339984">David Altherr</a>'],
     'photo_reference': 'CmRaAAAA5NHoAtwbhZDQlxQPcAJgu_AOppMe8J1SXL2O4PlUZmu30hK2WnGTAzYnWx8HmatwNhPp470g-tDeRnV3E8T5-iDF2_KwDj1Tk09gvq0Lodr_JywWpD4quQZzVGm1cwu2EhBB8EOJ-GRrr6Go_YrZalsxGhSKVsKBz4xpI4jwST2BZ5wYTi0glw',
     'width': 4608}],
   'place_id': 'ChIJ_f7qQlVDkFQRtzfcx8TZ05k',
   'plus_code': {'compound_code': 'FM2R+3F Seattle

In [6]:
df_airport = pd.DataFrame(getgeo(airport,r".*-Tacoma.*","airport"))
df_airport

,category,name,location
0,airport,Seattle-Tacoma International Airport,"{'type': 'Point', 'coordinates': [-122.3088165..."


In [7]:
cafe = getfromGooleApi(Querys(100000,"cafe","Starbucks",None))

200


In [8]:
df_cafe = pd.DataFrame(getgeo(cafe,r".*Starbucks.*","Starbucks"))
display(df_cafe.head(),df_cafe.shape)


,category,name,location
0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.440969,..."
1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.321584,..."
2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.198263,..."
3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.335842,..."
4,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.3241632..."


(20, 3)

In [9]:
searching = pd.concat([df_airport,df_cafe],axis=0)
searching

,category,name,location
0,airport,Seattle-Tacoma International Airport,"{'type': 'Point', 'coordinates': [-122.3088165..."
0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.440969,..."
1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.321584,..."
2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.198263,..."
3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.335842,..."
4,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.3241632..."
5,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.2986832..."
6,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.342583,..."
7,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.28294, ..."
8,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.31643, ..."


In [10]:
def coordinates_map(x):
    coor = x.location
    if type(coor)==dict:
        return {
        "latitude":float(coor["coordinates"][1]),
        "longitude":float(coor["coordinates"][0])
        }
    else:
        return("No coordinates")

In [11]:
searching_place = searching.apply(coordinates_map,axis=1,result_type="expand")
searching_place

,latitude,longitude
0,47.450250,-122.308817
0,47.206847,-122.440969
1,47.608774,-122.321584
2,47.616457,-122.198263
3,47.489000,-122.335842
4,47.609577,-122.324163
5,47.374516,-122.298683
6,47.610021,-122.342583
7,47.662940,-122.282940
8,47.314799,-122.316430


In [12]:
searching = pd.concat([searching,searching_place],axis=1)
searching

,category,name,location,latitude,longitude
0,airport,Seattle-Tacoma International Airport,"{'type': 'Point', 'coordinates': [-122.3088165...",47.450250,-122.308817
0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.440969,...",47.206847,-122.440969
1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.321584,...",47.608774,-122.321584
2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.198263,...",47.616457,-122.198263
3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.335842,...",47.489000,-122.335842
4,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.3241632...",47.609577,-122.324163
5,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.2986832...",47.374516,-122.298683
6,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.342583,...",47.610021,-122.342583
7,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.28294, ...",47.662940,-122.282940
8,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.31643, ...",47.314799,-122.316430


In [15]:
searching.to_json("./input/searching.json",orient="records")
searching.to_csv("./input/searching.csv")